## 직방 서비스 원룸 데이터 수집

- 복잡한 데이터 수집 방법
- 동이름 -> 매물정보(데이터 프레임) 

- 절차  
    - 동이름 -> 위도, 경도
    - 위도, 경도 -> geohash code(지역범위)
    - geohash -> 매물 아이디
    - 매물 아이디 -> 매물 정보

한글은 인코딩이 되어져있으므로 디코딩을 해서 확인해야한다.  
https://meyerweb.com/eric/tools/dencoder/

In [2]:
import requests
import pandas as pd
import json

### 1. 동이름 -> 위도, 경도

In [6]:
query = "망원동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸"
response = requests.get(url)
datas = response.json()["items"][0]
lat = datas["lat"]
lng = datas["lng"]

print(lat, lng)

37.556785583496094 126.9013442993164


### 2. 위도, 경도 -> geohash code(지역범위)

In [7]:
!pip install geohash2

Using legacy 'setup.py install' for geohash2, since package 'wheel' is not installed.
    Running setup.py install for geohash2: started
    Running setup.py install for geohash2: finished with status 'done'


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [8]:
import geohash2

In [12]:
# precision : 영역  값이 커질수록 영역이 작아짐
code = geohash2.encode(lat, lng, precision = 5)
code

'wydjx'

### 3. geohash -> 매물 아이디

In [28]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydjx&needHasNoFiltered=true&rent_gteq=0&sales_type_in=%EC%A0%84%EC%84%B8|%EC%9B%94%EC%84%B8&service_type_eq=%EC%9B%90%EB%A3%B8"
response = requests.get(url)
response

<Response [200]>

In [29]:
datas = response.json()["items"]
datas

[{'lat': 37.52961609164899, 'lng': 126.89696624101414, 'item_id': 30462357},
 {'lat': 37.529591551247805, 'lng': 126.8975796456714, 'item_id': 30524930},
 {'lat': 37.52960330065166, 'lng': 126.89758293588608, 'item_id': 30629925},
 {'lat': 37.52908470117425, 'lng': 126.89762112462735, 'item_id': 30681505},
 {'lat': 37.52962747652811, 'lng': 126.89132500624517, 'item_id': 30494154},
 {'lat': 37.529782431627254, 'lng': 126.8979400544405, 'item_id': 30351843},
 {'lat': 37.52978627834638, 'lng': 126.89733211493633, 'item_id': 30620037},
 {'lat': 37.52938426763489, 'lng': 126.9066126270265, 'item_id': 30218399},
 {'lat': 37.53058250481656, 'lng': 126.90761618908368, 'item_id': 30673258},
 {'lat': 37.531066078437085, 'lng': 126.90759829772804, 'item_id': 30673399},
 {'lat': 37.53125463053132, 'lng': 126.90756018047838, 'item_id': 30355099},
 {'lat': 37.53125776140002, 'lng': 126.9069626843332, 'item_id': 30444123},
 {'lat': 37.53127913107772, 'lng': 126.90756300504415, 'item_id': 30466344},


In [20]:
ids = [data["item_id"] for data in datas]
ids[:3]

[30462357, 30524930, 30629925]

In [23]:
# 리스트 컴프리헨션 : 간단한 리스트 데이터를 만들 때 사용
d1 = range(10)
print(list(d1))

# 홀수 데이터만 제곱해서 리스트 만들어 출력
result = [data**2 for data in d1 if data % 2]
print(result)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 9, 25, 49, 81]


### 4. 매물 아이디 -> 매물 정보

In [31]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain" : "zigbang",
    "withCoalition" : "true",
    "item_ids" : ids,
}


response = requests.post(url, params)
response

<Response [200]>

In [33]:
items = response.json()["items"]
item_df = pd.DataFrame(items)
item_df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
914,None,30455640,https://ic.zigbang.com/ic/items/30455640/1.jpg,전세,전세,28000,0,33.58,"{'m2': 33.58, 'p': '10.2'}","{'m2': 23.98, 'p': '7.3'}",...,False,True,빌라,[추천],서울시 마포구 중동,None,None,3,2022-02-03T12:00:40+09:00,False
915,None,30687168,https://ic.zigbang.com/ic/items/30687168/1.jpg,월세,월세,5000,60,41.36,"{'m2': 41.36, 'p': '12.5'}","{'m2': 36.36, 'p': '11'}",...,False,True,빌라,[],서울시 마포구 중동,None,None,8,2022-02-18T14:00:36+09:00,False


In [35]:
# 전체 다 보는 방법
pd.options.display.max_columns = 30

In [36]:
item_df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
914,None,30455640,https://ic.zigbang.com/ic/items/30455640/1.jpg,전세,전세,28000,0,33.58,"{'m2': 33.58, 'p': '10.2'}","{'m2': 23.98, 'p': '7.3'}",None,None,6,6,6,🔶🔶마포 저렴하고 전망 좋은 복층 쓰리룸 탑층🔶🔶,None,05,마포구 중동,"{'lat': 37.5699383839491, 'lng': 126.908875732...",False,True,빌라,[추천],서울시 마포구 중동,None,None,3,2022-02-03T12:00:40+09:00,False
915,None,30687168,https://ic.zigbang.com/ic/items/30687168/1.jpg,월세,월세,5000,60,41.36,"{'m2': 41.36, 'p': '12.5'}","{'m2': 36.36, 'p': '11'}",None,None,3,3,7,🟥🟨🟩 깔끔한 분위기~! 넓은 신축형 복층 원룸,None,04,마포구 중동,"{'lat': 37.56946245718027, 'lng': 126.90947401...",False,True,빌라,[],서울시 마포구 중동,None,None,8,2022-02-18T14:00:36+09:00,False


In [37]:
item_df.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new'],
      dtype='object')

In [38]:
columns = [
    'item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 
    'floor', 'building_floor', 'title', 'room_type', 'address', 
    'service_type', 'address1', 'manage_cost', 'reg_date', 'is_new'
]

In [44]:
result_df = item_df[columns]
result_df = result_df[result_df["address"].str.contains(query)]
result_df = result_df.reset_index(drop=True)
result_df

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30483493,전세,23000,0,18.99,6,6,"🦋합정역,신축1.5룸가성비짱,한강공원도보,보증보험OK",02,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-04T15:40:23+09:00,False
1,30608827,전세,23000,0,42.98,6,6,방1개 별도거실 주방 화장실 다용도실 분리형원룸 입니다,04,마포구 망원동,빌라,서울시 마포구 망원동,7,2022-02-12T16:27:27+09:00,False
2,30578522,월세,12000,10,12.21,3,5,💥LH 가능 깨끗한 방💥,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T18:54:14+09:00,False
3,30726074,월세,12000,10,13.22,3,5,🎶 [LH 가능!] 🎶 [깔끔한 방!],01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T11:45:06+09:00,True
4,30340227,월세,1000,49,23.14,3,5,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,30573911,전세,37000,0,36.17,2,6,마포구청역 3분.전세대출가능.무융자.신축급 컨디션 투룸~!,04,마포구 망원동,빌라,서울시 마포구 망원동,4.3,2022-02-10T13:55:31+09:00,False
90,30606067,전세,37000,0,39.67,2,6,🌷 대출가능! 투룸! 깔끔한 인테리어! 6호선! 🌷,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-16T18:36:13+09:00,False
91,30713445,월세,5000,30,19.83,3,4,🎢마포구청역5분 신축3년차🎢,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T13:46:30+09:00,True
92,30594723,월세,200,55,16.53,1,3,💖1.5층 단기방! 마포구청역 3분!💖,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-14T09:49:15+09:00,False


In [46]:
result_df[(result_df["deposit"] <= 10000) & (result_df["rent"] <= 100)]

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
4,30340227,월세,1000,49,23.14,3,5,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
5,30691402,월세,5000,90,33.06,4,4,🌈여성1인전용📢대출가능👑망원역6분🌈테라스💖,02,마포구 망원동,원룸,서울시 마포구 망원동,10,2022-02-18T12:12:34+09:00,False
6,30604487,전세,9300,0,15.00,2,5,망리단길 인접 풀옵션 원룸,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-02-21T02:39:16+09:00,True
15,30687981,월세,1000,85,49.59,3,4,⭕망원역세권 남향투룸 내부깔끔⭕,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-18T09:50:36+09:00,False
16,30488407,월세,500,43,19.83,옥탑방,3,🤩이넓은테라스가다내꺼🌞,02,마포구 망원동,원룸,서울시 마포구 망원동,2,2022-02-16T17:10:20+09:00,False
20,30620775,월세,3000,100,52.89,3,5,⭕망원동 한강공원인근 투룸월세⭕,04,마포구 망원동,빌라,서울시 마포구 망원동,2,2022-02-14T10:25:58+09:00,False
26,30592602,월세,2000,55,16.53,3,5,🎉이쁜 인테리어 풀옵 원룸🎉,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T12:53:03+09:00,False
30,30522407,월세,10000,60,42.84,2,5,중기청80% 통베란다 넓고 전대가능한 투룸 월세,04,마포구 망원동,빌라,서울시 마포구 망원동,3,2022-02-07T14:39:54+09:00,False
34,30707009,월세,3000,100,59.50,2,3,⭕망원시장안 쓰리룸 월세 주거용사무실가능⭕,05,마포구 망원동,빌라,서울시 마포구 망원동,0,2022-02-19T13:02:03+09:00,False
36,30728052,월세,5000,50,46.28,2,4,*!* 망원시장 올수리된 투룸 전월세 *!^,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-21T12:52:41+09:00,True


In [52]:
def oneroom(address):

    # 위도 경도 얻기
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸"
    response = requests.get(url)
    datas = response.json()["items"][0]
    lat = datas["lat"]
    lng = datas["lng"]

    code = geohash2.encode(lat, lng, precision = 5)

    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    datas = response.json()["items"]

    ids = [data["item_id"] for data in datas]

    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain" : "zigbang",
        "withCoalition" : "true",
        "item_ids" : ids[:998],
    }


    response = requests.post(url, params)
    items = response.json()["items"]
    item_df = pd.DataFrame(items)
    columns = [
        'item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 
        'floor', 'building_floor', 'title', 'room_type', 'address', 
        'service_type', 'address1', 'manage_cost', 'reg_date', 'is_new'
    ]
    result_df = item_df[columns]
    result_df = result_df[result_df["address"].str.contains(address)]
    result_df = result_df.reset_index(drop=True)
    

    return result_df


In [54]:
oneroom("남구 삼산동")

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30696236,월세,200,28,33.06,3,4,"✨❤️넓은,분리형💚대박💚풀옵션💚깔끔한방!❤️✨",02,남구 삼산동,원룸,울산시 남구 삼산동,5,2022-02-18T15:45:57+09:00,False
1,30700948,월세,200,28,29.80,2,4,🚖터미널❣백화점 도보 5분🚖편의시설 최적의 조건👀,02,남구 삼산동,원룸,울산시 남구 삼산동,5,2022-02-18T18:56:14+09:00,False
2,30626300,월세,100,26,29.75,3,5,❤남향!!❤완전깨끗해요❤😆가스비포함❤😆이쁜방!!❤,01,남구 삼산동,원룸,울산시 남구 삼산동,7,2022-02-14T13:06:47+09:00,False
3,30636665,월세,300,30,42.98,3,4,🟥리모델링분리형원룸🔴방크기실화❓🅰최저가월세,02,남구 삼산동,원룸,울산시 남구 삼산동,5,2022-02-15T13:05:09+09:00,False
4,30733056,월세,500,55,56.20,3,4,🟥실매물🟧큰방2+거실🟨완전깔끔🟩주차편리🟪,04,남구 삼산동,빌라,울산시 남구 삼산동,5,2022-02-21T15:48:07+09:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,30714655,월세,200,30,36.36,3,4,⊱🌸⊰ ◾현백도보◾1.5R◾풀옵션◾실사진 ⊱🌸⊰,02,남구 삼산동,원룸,울산시 남구 삼산동,5,2022-02-19T20:13:41+09:00,True
77,30732642,월세,200,30,36.36,2,4,💥👍💥신축급!!깔끔한느낌!!주차편리!!넓은방!!,02,남구 삼산동,원룸,울산시 남구 삼산동,5,2022-02-21T15:33:52+09:00,True
78,30682355,월세,300,40,50.02,2,4,💖방2개💖깨끗하고깔끔함💖풀옵션💖채광굿,04,남구 삼산동,빌라,울산시 남구 삼산동,5,2022-02-21T11:55:16+09:00,True
79,30695283,월세,300,50,56.20,2,4,⊱🌸⊰ ◾초특가 방2+거실◾풀옵션/ /실사진,04,남구 삼산동,빌라,울산시 남구 삼산동,5,2022-02-18T15:05:17+09:00,False
